In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import xlrd
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import random
import omdb
from omdb import Client
omdb.set_default('apikey', '18913aaf')
client = omdb.Client(apikey='18913aaf')
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [3]:
#this function takes a file name for an excel spreadsheet saved to my desktop, converts the data into a pandas data frame,then 
# appendss the elements from the inputed column to a list which is returned 
    #fileName: name of xls file saved to desktop
    #column: column of dataframe that we want to pull data from 
def excelToList(fileName,column):
    xls_file = pd.ExcelFile('C:/Users/adam5_000/Desktop/'+ fileName + '.xls')
    df = xls_file.parse('Sheet1')
    lis = [df[column][i] for i in range(0,len(df))]
    return lis

In [4]:
#this function goes through the original dirty data, identifies the FF52 films, theater rentals, or one-off events, and returns
#a new list of film titles without those movies. Also returned is a list containing the number admissions for our new list of 
#movies where the indices of the two lists correspond to each other
    #fileName: name of xls file saved on desktop
def removeFF2(fileName):
    titles = excelToList(fileName,'Film Title')
    avg_show = excelToList(fileName,'Avg/Show')
    gross = excelToList(fileName,'Gross')
    admiss = excelToList(fileName,'Admissions')
    ff52 = []
    movies = []
    labels = []
    number_shows = [1,2,3,4,5,6,7]
    
    for i in range(len(titles)):
        for k in number_shows:
            if int(k*(avg_show[i]))==int(gross[i]):
                ff52.append(titles[i])
                
    for j in range(len(titles)):
        if titles[j] not in ff52:
            movies.append(titles[j])
            labels.append(admiss[j])
                
    return movies,labels

In [5]:
#this function takes a list of movies and identifies those film titles which are spelled wrong, The function returns a list 
#of the films titles with spelling errors
    #lis: list of film titles
def problemMovies(lis):
    problems = []
    for i in range(len(lis)):
        mv = omdb.get(title=lis[i])
        if len(mv)==0:
            problems.append((i,lis[i]))
    return problems

In [6]:
#this function is used to make the spelling correction for those films found to have spelling errors. The user will be asked
#to input a corrected film title, this will then replace the incorrect film title in the list of movies 
    #lis1: list of movies with spelling errors
    #lis2: list of all movies 
def makeChanges(lis1,lis2):
    for i in range(len(lis1)):
        p = input('Correct '+lis1[i][1]+' film title to: ')
        ind = lis1[i][0]
        if len(omdb.get(title=p)) !=0:
            lis2[ind] = p
    return lis2

In [7]:
#this function will take a list of movies, find the genre(s) for each movies and add them to a list which is returned
    #lis: list of film titles 
def returnGenres(lis):
    totalGenres = []
    for i in range(len(lis)):
        genres = omdb.get(title=lis[i])['genre']
        genreList = genres.split(',')
        totalGenres+=genreList
    for j in range(len(totalGenres)):
        totalGenres[j] = totalGenres[j].lstrip()
    return list(set(totalGenres))        

In [8]:
#this function will take a list of film titles, make a list of the production companies for those movies. Certain words are
#removed from each production company name for consistency purposes. Then a count is made of the frequency of each production 
#company and returned is a list of the production companies that showed up more than one time. 
def returnProduction(lis):
    productionList = []
    topDic = []
    keys = ['& broad green pictures','films','film','pictures','picture','corporation','studios','media','releasing','distribution','cinema',
            'laboratories','entertainment','llc','releasing','the','co.','co./mgm','company']
    
    key_changes = ['millennium','warner brors','sony /columbia']
    key_replaces = ['millenium','warner bros.','sony/columbia']
    
    for i in range(len(lis)):
        production = omdb.get(title=lis[i])['production']
        productionList.append(production.lower())
    
    for j in range(len(productionList)):
        for l in keys:
            if l in productionList[j]:
                productionList[j] = productionList[j].replace(l,'')

        productionList[j] = productionList[j].strip()
        if '  ' in productionList[j]:
            productionList[j] = productionList[j].replace('  ',' ')
            
        if productionList[j] in key_changes:
            ind = key_changes.index(productionList[j])
            productionList[j] = key_replaces[ind]
            
    prod_dict = {k:productionList.count(k) for k in productionList}
    prod_dict_Words = list(prod_dict)
    
    
    for l in range(len(prod_dict_Words)):
        if (prod_dict[prod_dict_Words[l]]>1):
            topDic.append(prod_dict_Words[l])
            
    return topDic

In [9]:
#this function takes a list of film titles, creates a list of all the directors from each film, we then count the frequency for 
#each director, returned is a list of directors who showed up more than once. 
def returnDirectors(lis):
    totalDirectors = []
    topDirectors = []
    for i in range(len(lis)):
        director = omdb.get(title=lis[i])['director']
        directorList = director.split(',')
        totalDirectors+=directorList
    
    for j in range(len(totalDirectors)):
        totalDirectors[j] = totalDirectors[j].lstrip()
    
    count = {k:totalDirectors.count(k) for k in totalDirectors}
    names = list(count)
    for l in range(len(names)):
        if(count[names[l]]>1):
            topDirectors.append(names[l])
            
    return topDirectors 

In [10]:
#this function takes a list of film titles, creates a list of all the actors from each film, counts the frequency of each actor
# in the list, returned is a list of actors who showed up more than once. 
def returnActors(lis):
    totalActors = []
    topActors = []
    for i in range(len(lis)):
        actors = omdb.get(title=lis[i])['actors']
        actorList = actors.split(',')
        totalActors+=actorList
        
    for j in range(len(totalActors)):
        totalActors[j] = totalActors[j].lstrip()
            
    count = {k:totalActors.count(k) for k in totalActors}
    names = list(count)
    for l in range(len(names)):
        if(count[names[l]]>1):
            topActors.append(names[l])
    
    return topActors

In [11]:
#this function takes a list of film titles, retrieves the plot for each film. we then analyze every plot by (1.) removing stop
#words, (2.) remove punctuation '.','-', the words are then stemmed. We then find the frequency of each word from the plots
#and return a list containing those plot words that showed up 5 times or more. 
def keyPlotWords(lis):
    keyWs = []
    newL = []
    topWords = []
    s = set(stopwords.words('english'))
    for i in range(len(lis)):
        plot = omdb.get(title=lis[i])['plot']
        plotList = filter(lambda w: not w in s, plot.split())
        keyWs+=plotList
        
    for j in range(len(keyWs)):
        if('.' in keyWs[i]):
            keyWs[j] = keyWs[j].replace('.','')
        if('-' in keyWs[i]):
            keyWs[j] = keyWs[j].replace('-','')
    
    newL = [PorterStemmer().stem(keyWs[k]) for k in range(len(keyWs))]
    
    my_dict = {l:newL.count(l) for l in newL}
    my_dict_Words = list(my_dict)
    
    for m in range(len(my_dict_Words)):
        if (my_dict[my_dict_Words[m]]>=5):
            topWords.append(my_dict_Words[m])
        
    return topWords

In [12]:
#this function takes two lists of words, find the words that show up in both lists, then returns each list with the words that 
#showed up in both lists removed. 
def samePlotWords(lis1,lis2):
    same = (set(lis1) & set(lis2))
    return list(set(lis1)-same),list(set(lis2)-same)

In [13]:
#this function takes a list of film titles, retrieves the writer(s) for each film and adds them to a list. We then count the 
#frequency for each writer and return a list of the writers who showed up more than once. 
def returnWriters(lis):
    writerList = []
    topWriters = []
    for i in range(len(lis)):
        writers = omdb.get(title=lis[i])['writer']
        writers = writers.split(',')
        
        for j in range(len(writers)):
            writers[j] = writers[j].lstrip()
            if '(' in writers[j]:
                ind = writers[j].find('(')
                writerList.append(writers[j][0:ind-1])
            else:
                writerList.append(writers[j])
        
    count = {k:writerList.count(k) for k in writerList}
    names = list(count)
    for l in range(len(names)):
        if(count[names[l]]>1):
            topWriters.append(names[l])
            
    return topWriters

In [14]:
#this function normalizes given data
def normalized(X):
    normal = np.zeros((len(X),len(X[0,:])))
    for i in range(len(X[0,:])):
        mu = np.mean(X[:,i])
        sigma = np.std(X[:,i])
        normal[:,i] = (X[:,i]-mu)/sigma
    return normal

In [15]:
#this function takes a list of film titles, a list of #admissions for each film, and the cutoff admission # seperating the 
#two classes. The films with admissions below the cutoff are labeled 0 and above the threshold labeled 1. Also created is 
#a list containing all the film titles with label 0 and a list containg all the film titles with label 1. Returned is the list
#of labels, the list of film titles with label 0, and a list of film titles with label 1. 
def createLabels(movies,admiss,cutoff):
    labels = []
    mvsl0 = []
    mvsl1 = []
    for i in range(len(admiss)):
        if admiss[i]<=cutoff:
            labels.append(float(0))
            mvsl0.append(movies[i])
        else:
            labels.append(float(1))
            mvsl1.append(movies[i])

    return labels,mvsl0,mvsl1

In [16]:
#this function will take (1.) a list of movies,(2.) a list of all the movies labeled y=0, and (3.) a list of all the
#movies labeled y=1 as inputs and returns an mxn array where m is the number of movies and n is the number
#of features. The features include: imdb_rating, metascore, runtime, #imdb_votes, #writers, genre(dummy variable),month(dummy)
#mpaa(dummy),production company(dummy),actors(dummy),directors(dummy),plot keywords(y=0)(dummy), plot keywords(y=1)(dummy)
#writers(dummy)

def getData(totalMovies,MVsL1,MVsL2):    
    
    #list of all genres 
    genList = returnGenres(totalMovies)
    #number of genres
    numGens = len(genList)
    
    #list of all production companies that showed up more than once 
    prodsList = returnProduction(totalMovies)
    #number of production companies that showed up more than once
    numProds = len(prodsList)
    
    #list of all directors who showed up more than once 
    listD =  returnDirectors(totalMovies)
    #number of directors who showed up more than once 
    numDirec = len(listD)
    
    #list of all actors who showed up more than once 
    listA = returnActors(totalMovies)
    #number of actors
    numAct = len(listA)

    #list of top words in plots from movies with label y=0
    topPlotWordsL1 = keyPlotWords(MVsL1)
    #list of top words in plots from movies with label y=1
    topPlotWordsL2 = keyPlotWords(MVsL2)
    
    #list of top plot words y=0,y=1 without words that show up in both lists
    finalKey1 = samePlotWords(topPlotWordsL1,topPlotWordsL2)[0]
    finalKey2 = samePlotWords(topPlotWordsL1,topPlotWordsL2)[1]
    #number of plot keywords (y=0)
    numKey1 = len(finalKey1)
    #number of plot keywords (y=1)
    numKey2 = len(finalKey2)
    
    #list of all the writers who showed up in more than one movie 
    topWriters = returnWriters(totalMovies)
    #number of writers
    numWrit = len(topWriters)
    
    #this will serve as the number of columns for our X-matrix, 5 numerical features+7 categorical (dummy) features
    numFeatures = 5+numGens+numProds+numDirec+numAct+numKey1+numKey2+numWrit
    
    #this is the mxn matrix that will be filled with movie data and returned 
    X = np.zeros((len(totalMovies),numFeatures))
     
    #this will loop over all the movies in the data set provided and pull out all feature data     
    for i in range(len(totalMovies)):
        lis = []
        mv = omdb.get(title=totalMovies[i])

        lis.append(float(mv['imdb_rating']))
        
        metascore = mv['metascore']
        if metascore == 'N/A':
            lis.append(float(mv['imdb_rating'])*10)
        else:
            lis.append(float(mv['metascore']))
        
        runtime = (mv['runtime'])
        spaceInds = []
        for j in range(len(runtime)):
            if(runtime[j] == ' '):
                spaceInds.append(j)
        lis.append(float(runtime[0:spaceInds[0]]))
        
        #lis.append(len(countWriters(totalMovies[i])))
        lis.append(len(mv['writer'].split(',')))
        
        numVotes = (mv['imdb_votes'])
        newNum = numVotes.replace(',','')
        lis.append(float(newNum))
     
        genre = mv['genre']
        for n in range(len(genList)):
            if genList[n] in genre:
                lis.append(float(1))
            else:
                lis.append(float(0))
        
        date = (mv['released'])
        months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        for k in range(len(months)):
            if months[k] in date:
                lis.append(float(1))
            else:
                lis.append(float(0))
            
        mpaa = (mv['rated'])
        ratings = ['G','PG','PG-13','R','N/A','UNRATED']
        for l in range(len(ratings)):
            if ratings[l] in mpaa:
                lis.append(float(1))
            else:
                lis.append(float(0))
                
        production = mv['production']
        production = production.lower()
        for o in range(len(prodsList)):
            if prodsList[o] in production:
                lis.append(float(1))
            else:
                lis.append(float(0))
        
        director = mv['director']
        for p in range(len(listD)):
            if listD[p] in director:
                lis.append(float(1))
            else:
                lis.append(float(0))
                
        actors = mv['actors']
        for q in range(len(listA)):
            if listA[q] in actors:
                lis.append(float(1))
            else:
                lis.append(float(0))
        
        stemWords = []     
        s = set(stopwords.words('english'))
        plot = mv['plot']
        plotWords = filter(lambda w: not w in s, plot.split())

        for r in range(len(plotWords)):
            if('.' in plotWords[r]):
                plotWords[r] = plotWords[r].replace('.','')
            if('-' in plotWords[r]):
                plotWords[r] = plotWords[r].replace('-','')
            stemWords.append(PorterStemmer().stem(plotWords[r]))
            
        for s in range(len(finalKey1)):
            if(finalKey1[s] in stemWords):
                lis.append(float(1))
            else:
                lis.append(float(0))
                
        for t in range(len(finalKey2)):
            if(finalKey2[t] in stemWords):
                lis.append(float(1))
            else:
                lis.append(float(0))
        
        writers = mv['writer']
        for u in range(len(topWriters)):
            if topWriters[u] in writers:
                lis.append(float(1))
            else:
                lis.append(float(0))    
            
        #add all feature data found above for a given movie and put into X
        #one row of X corresponds to one movie 
        for m in range(0,X.shape[1]):
            X[i,m] = lis[m]
        
    return X        

In [28]:
#Now we have all the functions neccessary to gather our feature data and the function getData() which will call all the 
#releveant functions and convert the data into a matrix capable of being analyzed by ML algorithms 

In [17]:
#Remove FF52 movies from dirty data and return labels, and film titles
labels = removeFF2('Dirt')[1]
movies = removeFF2('Dirt')[0]

In [18]:
#find all movies that have len==0, indicating spelling error 
lenZero = problemMovies(movies)

In [20]:
#correct all the misspelled film titles 
makeChanges(lenZero,movies)

Correct BEASTS OF SOUTHERN WILD film title to: 'Beasts of the Southern Wild'
Correct BEST EXOTIC MARIGOLD HOTE film title to: 'The Best Exotic Marigold Hotel'
Correct BILL CUNNINGHAM'S NEW YOR film title to: 'Bill Cunningham New York'
Correct CELESTE AND JESSE film title to: 'Celeste and Jesse Forever'
Correct CITY OF YOUR FINAL DESTIN film title to: 'The City of Your Final Destination'
Correct COCO AND IGOR film title to: 'Coco Chanel & Igor Stravinsky'
Correct EAT THAT QUESTION: ZAPPA film title to: 'Eat That Question: Frank Zappa in His Own Words'
Correct ELVIS AND NIXON film title to: 'Elvis & Nixon'
Correct EMBRACE THE SERPENT film title to: 'Embrace of the Serpent'
Correct EXTREMELY LOUD AND INCRED film title to: 'Extremely Loud & Incredibly Close'
Correct FAR FROM MADDING CROWD film title to: 'Far from the Madding Crowd'
Correct GINGER AND ROSA film title to: 'Ginger & Rosa'
Correct GIRL WHO KICKED THE HORNE film title to: "The Girl Who Kicked the Hornets' Nest"
Correct GIRL WIT

[u'100 YEAR OLD MAN',
 u'12 YEARS A SLAVE',
 u'127 HOURS',
 u'20TH CENTURY WOMEN',
 42.0,
 u'45 YEARS',
 u'5 FLIGHTS UP',
 u'50/50',
 71.0,
 u'99 HOMES',
 u'A BETTER LIFE',
 u'A BIGGER SPLASH',
 u'A BRILLIANT YOUNG MIND',
 u'A DANGEROUS METHOD',
 u'A FIVE STAR LIFE',
 u'A LATE QUARTET',
 u'A MAN CALLED OVE',
 u'A MOST WANTED MAN',
 u'A QUIET PASSION',
 u'A SEPARATION',
 u'A SINGLE MAN',
 u'A WALK IN THE WOODS',
 u'ABOUT TIME',
 u'ABSOLUTELY FABULOUS',
 u'ADJUSTMENT BUREAU',
 u'ADMISSION',
 u'ALBERT NOBBS',
 u'ALICE IN WONDERLAND',
 u'ALL GOOD THINGS',
 u'ALL IS LOST',
 u'ALLIED',
 u'ALOHA',
 u'AMERICAN HUSTLE',
 u'AMERICAN SNIPER',
 u'AMOUR',
 u'AMY',
 u'AN EDUCATION',
 u'AN INCONVENIENT SEQUEL',
 u'ANNA KARENINA',
 u'ANOTHER EARTH',
 u'ANOTHER YEAR',
 u'ANTHROPOID',
 u'ARBITRAGE',
 u'ARGO',
 u'ARRIVAL',
 u'AT ANY PRICE',
 u'AUGUST OSAGE COUNTY',
 u'AUSTENLAND',
 u'BABIES',
 u"BARNEY'S VERSION",
 u'BATTLE OF THE SEXES',
 'Beasts of the Southern Wild',
 u'BEATRIZ AT DINNER',
 u'BEFORE M

In [59]:
#save corrected film titles in a datafram and save all the clean data in an excel sheet titled, 'Clean.xls'
df = pd.DataFrame(movies,labels)
writer=pd.ExcelWriter('C:/Users/adam5_000/Desktop/Clean.xls')
df.to_excel(writer,'Sheet1')
writer.save()

In [91]:
#Retrieve the cleaned admissions data 
cleanLabels = excelToList('Clean','Admissions')
#Retrieve the cleaned film title data 
cleanMovies = excelToList('Clean','Film Titles')

In [92]:
#labels given cutoff #admissions = 650
labels = createLabels(cleanMovies,cleanLabels,650)[0]
#all the films with label y=0
mvl0 = createLabels(cleanMovies,cleanLabels,650)[1]
#all the films with label y=1
mvl1 = createLabels(cleanMovies,cleanLabels,650)[2]

In [93]:
#zip together movie titles and corresponding label 
zipDat = list(zip(cleanMovies,labels))
#randomly shuffle the new zipped list 
random.shuffle(zipDat)
#unzip the shuffled list while keeping corresponding movie titles and labels matched  
#movie titles go back into movieDat and labels go back into movieLabels 
movieDat, movieLabels = zip(*zipDat)
#now we have our data randomly shuffled
print movieDat[0:5], movieLabels[0:5]

(u'THE JUDGE', u'WORDS AND PICTURES', u'RUSH', u'PATERSON', u'St. Vincent') (1.0, 0.0, 0.0, 0.0, 1.0)


In [94]:
#get X-matrix 
totalDat = getData(movieDat,mvl0,mvl1)

In [95]:
#normalize the numerical features for the entire dataset (imdb_rating,metascore,runtime,#imdb_votes,#writers)
totalNorm = normalized(totalDat[:,0:5])
#add the normalized data back to the dataset
totalDat = np.append(totalNorm,totalDat[:,5:totalDat.shape[1]],axis=1) 

In [233]:
cvScores = cross_val_score(LogisticRegression(C=1),totalDat,movieLabels,cv=5)

In [1]:
print("5-Cross Validation scores: {}".format(cvScores)) 

NameError: name 'cvScores' is not defined

In [235]:
print('Mean 5-CV Score: {:.2f}%').format(cvScores.mean()*100)

Mean 3-CV Score: 68.43%


In [102]:
#split the training set which is 80% of entire dataset
trainMovies = totalDat[0:404,:]
trainLabels = movieLabels[0:404]
#split the test set which is 20% of the entire dataset
testMovies = totalDat[404:len(totalDat),:]
testLabels = movieLabels[404:len(movieLabels)]

In [145]:
#this function takes as inputs train data, test data, a given model. The function will evaluate the given model for given a range
#of C values (greater than  1) which is the regularization parameter. The function will output the highest test set accuracy
#s.t test acc. < train acc. and the place of the C term 
def accuracyReg(xtrain,ytrain,xtest,ytest,model):
    scores = np.zeros((150,2))
    for i in range(1,150):
        Model = model(C=i).fit(xtrain,ytrain)
        trainScore = Model.score(xtrain,ytrain)*100
        testScore = Model.score(xtest,ytest)*100
        if trainScore>=testScore:
            scores[i,0] = testScore
            scores[i,1] = i
    maxVal = max(scores[:,0])
    location = np.where(scores==maxVal)
    row = location[0][0]
    return [maxVal,row]

In [146]:
#this function takes as inputs train data, test data, a given model, and t which can be adjusted for tenths(10), hundreths(100), 
#thousandths(1000). The function will evaluate the given model for given a range of C values (less than  1) which is the 
#regularization parameter. The function will output the highest test set accuracy s.t test acc. < train acc. and the place 
#of the C term 

def accuracyRegD(xtrain,ytrain,xtest,ytest,model,t):
    scores = np.zeros((150,2))
    for i in range(9,0,-1):
        Model = model(C=(float(i)/t)).fit(xtrain,ytrain)
        trainScore = Model.score(xtrain,ytrain)*100
        testScore = Model.score(xtest,ytest)*100
        if trainScore>=testScore:
            scores[i,0] = testScore
            scores[i,1] = i
    maxVal = max(scores[:,0])
    location = np.where(scores==maxVal)
    row = location[0][0]
    return [maxVal,row]

In [148]:
#this function takes as inputs train data, test data, a given MLP model. The function will evaluate the given model for a given
#range of hidden layer sizes. The function will output the highest test set accuracy s.t test acc. < train acc.
#and the place of the hidden layer size term 
def accuracyMLP(xtrain,ytrain,xtest,ytest,model):
    scores = np.zeros((150,2))
    for i in range(1,150):
        Model = model(solver='adam',activation='relu',random_state=42,hidden_layer_sizes=[i]).fit(xtrain,ytrain)
        trainScore = Model.score(xtrain,ytrain)*100
        testScore = Model.score(xtest,ytest)*100
        if trainScore>=testScore:
            scores[i,0] = testScore
            scores[i,1] = i
    maxVal = max(scores[:,0])
    location = np.where(scores==maxVal)
    row = location[0][0]
    return [maxVal,row]

In [158]:
print accuracyReg(trainMovies,trainLabels,testMovies,testLabels,LogisticRegression)
print accuracyRegD(trainMovies,trainLabels,testMovies,testLabels,LogisticRegression,100)

[72.0, 2]
[76.0, 7]


In [156]:
LogReg =  LogisticRegression(C=.07,random_state=42).fit(trainMovies,trainLabels)
print("Training Score (sklearn): {:.2f}%".format(LogReg.score(trainMovies,trainLabels)*100))
print("Test Score (sklearn): {:.2f}%".format(LogReg.score(testMovies,testLabels)*100))

Training Score (sklearn): 80.69%
Test Score (sklearn): 76.00%


In [176]:
print accuracyMLP(trainMovies,trainLabels,testMovies,testLabels,MLPClassifier)

[70.0, 5]


In [200]:
neuralNet = MLPClassifier(solver='adam',activation='relu',random_state=42,hidden_layer_sizes=[5]).fit(trainMovies,trainLabels)
print("Training Score (sklearn): {:.2f}%".format(neuralNet.score(trainMovies,trainLabels)*100))
print("Test Score (sklearn): {:.2f}%".format(neuralNet.score(testMovies,testLabels)*100))

Training Score (sklearn): 95.54%
Test Score (sklearn): 70.00%


In [166]:
print accuracyReg(trainMovies,trainLabels,testMovies,testLabels,LinearSVC)
print accuracyRegD(trainMovies,trainLabels,testMovies,testLabels,LinearSVC,1000)

[76.0, 5]


In [169]:
SupVecClass = LinearSVC(C=5).fit(trainMovies,trainLabels)
print("Training Score (sklearn): {:.2f}%".format(SupVecClass.score(trainMovies,trainLabels)*100))
print("Test Score (sklearn): {:.2f}%".format(SupVecClass.score(testMovies,testLabels)*100))

Training Score (sklearn): 100.00%
Test Score (sklearn): 67.00%


In [174]:
print accuracyReg(trainMovies,trainLabels,testMovies,testLabels,SVC)
print accuracyRegD(trainMovies,trainLabels,testMovies,testLabels,SVC,1)

[75.0, 8]


In [170]:
svc = SVC(C=8).fit(trainMovies,trainLabels)
print("Training Score (sklearn): {:.2f}%".format(svc.score(trainMovies,trainLabels)*100))
print("Test Score (sklearn): {:.2f}%".format(svc.score(testMovies,testLabels)*100))

Training Score (sklearn): 76.49%
Test Score (sklearn): 75.00%
